In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

os.listdir("dataset")

['Test', 'Train']

In [2]:
train_dir = "dataset/Train"
test_dir = "dataset/Test"


print("Train set:\n========================================")
num_Covid19_Positive = len(os.listdir(os.path.join(train_dir, 'Covid19 Positive')))
num_Covid19_Negative = len(os.listdir(os.path.join(train_dir, 'Covid19 Negative')))
Pneumonia = len(os.listdir(os.path.join(train_dir, 'Pneumonia')))
print(f"Covid19 Positive={num_Covid19_Positive}")
print(f"Covid19 Negative={num_Covid19_Negative}")
print(f"Pneumonia={Pneumonia}")

print("Test set:\n========================================")
print(f"Covid19 Positive={len(os.listdir(os.path.join(test_dir, 'Covid19 Positive')))}")
print(f"Covid19 Negative={len(os.listdir(os.path.join(test_dir, 'Covid19 Negative')))}")

print(f"Pneumonia={len(os.listdir(os.path.join(test_dir, 'Pneumonia')))}")



Train set:
Covid19 Positive=2531
Covid19 Negative=7134
Pneumonia=943
Test set:
Covid19 Positive=1085
Covid19 Negative=3058
Pneumonia=402


In [3]:
from keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

Using TensorFlow backend.


In [4]:

train = image_generator.flow_from_directory(train_dir, 
                                            batch_size=8, 
                                            shuffle=True, 
                                            class_mode='categorical',
                                            target_size=(64,64))



test = image_generator.flow_from_directory(test_dir, 
                                            batch_size=1, 
                                            shuffle=False, 
                                            class_mode='categorical',
                                            target_size=(64,64))

Found 10608 images belonging to 3 classes.
Found 4545 images belonging to 3 classes.


In [5]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

base_model = DenseNet121(input_shape=(180, 180, 3), include_top=False, weights='imagenet', pooling='avg')

base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 180, 180, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 186, 186, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 90, 90, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 90, 90, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [6]:
layers = base_model.layers
print(f"The model has {len(layers)} layers")

The model has 428 layers


In [13]:

#model = Sequential()
base_model = DenseNet121(include_top=False, weights='imagenet')
x = base_model.output

x = GlobalAveragePooling2D()(x)

predictions = Dense(3, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)
#model.add(base_model)
#model.add(GlobalAveragePooling2D())
#model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [11]:
from math import *

In [19]:
BATCH_SIZE=32

TRAINING_SIZE = 40000

VALIDATION_SIZE = 10000

# We take the ceiling because we do not drop the remainder of the batch
compute_steps_per_epoch = lambda x: int(ceil(1. * x / BATCH_SIZE))

steps_per_epoch = compute_steps_per_epoch(TRAINING_SIZE)
val_steps = 100

history = model.fit(x=train,
              epochs=100,
              steps_per_epoch=100,
              validation_data=test,
              
              validation_steps=val_steps,
              shuffle=True)

Epoch 1/100
 19/100 [====>.........................] - ETA: 37s - loss: 0.2537 - accuracy: 0.9123

KeyboardInterrupt: 